## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
import mlflow
mlflow.__version__

'1.26.1'

## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset. 

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

In [28]:
!python preprocess_data.py --raw_data_path ../data --dest_path ../data/output

In [29]:
import os

_, _, files = next(os.walk("../data/output"))
len(files)

4

## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

In [21]:
# Run script
!python train.py --data_path /Users/Jan/MLOps/mlops-zoomcamp/02-experiment-tracking/data/output

# Create client, get experiment, get run and access params
client = mlflow.tracking.MlflowClient(tracking_uri = 'sqlite:///taxidb.sqlite')
experiment = client.get_experiment_by_name('homework_2_taxi_experiment')
runs = client.search_runs(1)
len(runs[0].data.params)

2022/05/31 11:39:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/05/31 11:39:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

17

## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

In [22]:
# Run incomplete script to get error about required arg
!mlflow server --backend-store-uri sqlite:///taxidb.sqlite

Option 'default-artifact-root' is required when backend store is not local file based.
Usage: mlflow server [OPTIONS]
Try 'mlflow server --help' for help.

Error: Option 'default-artifact-root' is required when backend store is not local file based.


Running `mlflow server --backend-store-uri "sqlite:///taxidb.sqlite" --default-artifact-root "./artifacts"` in terminal

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

In [32]:
!python hpo.py

100%|█████████| 50/50 [07:23<00:00,  8.88s/trial, best loss: 6.6284257482044735]


In [34]:
best_run = client.search_runs(
  experiment_ids = 2, 
  run_view_type= mlflow.entities.ViewType.ACTIVE_ONLY,
  max_results=1,
  order_by=["metrics.rmse ASC"]
)[0]

best_run.data.metrics['rmse']

6.6284257482044735

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

In [44]:
!python register_model.py

ERROR in rec_eval
EXCEPTION <class 'AttributeError'> 'NoneType' object has no attribute 'uniform'
NODE
0 quniform
1   Literal{10}
2   Literal{50}
3   Literal{1}
Traceback (most recent call last):
  File "register_model.py", line 97, in <module>
    run(args.data_path, args.top_n)
  File "register_model.py", line 64, in run
    train_and_log_model(data_path=data_path, params=run.data.params)
  File "register_model.py", line 40, in train_and_log_model
    params = space_eval(SPACE, params)
  File "/Applications/anaconda3/lib/python3.8/site-packages/hyperopt/fmin.py", line 618, in space_eval
    rval = pyll.rec_eval(space, memo=memo)
  File "/Applications/anaconda3/lib/python3.8/site-packages/hyperopt/pyll/base.py", line 902, in rec_eval
    rval = scope._impls[node.name](*args, **kwargs)
  File "/Applications/anaconda3/lib/python3.8/site-packages/hyperopt/pyll/stochastic.py", line 49, in quniform
    draw = rng.uniform(low, high, size=size)
AttributeError: 'NoneType' object has no attrib